#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> MI: Luis Enrique Argota V.  </font>

##### <font color="#03997A"> Modulo 9 Natural Language Procesing </font>

##### <font color="#03997A"> Tarea 4 - Apredizaje Supervisado para Clasificación de Textos
##### <font color="#03997A"> Equipo 1:
- Sergio Ibarra Ramírez
- Yafte Aaron Flores Morales
- Esteban Bracamontes Orozco

</font>

In [247]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

from sklearn.preprocessing import MinMaxScaler

# 1. Preprocesamiento

In [248]:
corpus_train_esA = pd.read_csv('/content/train_es.tsv',delimiter='\t',encoding='utf-8')
corpus_dev_esA = pd.read_csv('/content/dev_es.tsv',delimiter='\t',encoding='utf-8')

### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)

In [249]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):
    file[file.columns[1]] = [clean_text(i) for i in file[file.columns[1]]]
    file.to_csv(namefile, sep='\t', encoding='utf-8', index=False)
    return file

def clean_text(text):
    text = text.lower()
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", " ", text)
    text=re.sub(pattern_URL, " ", text)

    text= remove_emoji(text)
    text= remove_stopwords(text)
    text=re.sub("\d+", "0", text)
    # text=re.sub("\d+", " ", text)

    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):
    stopwords=set(nltk.corpus.stopwords.words("spanish"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # no emoji
    return text

### Guardando el corpus ya procesado A

In [250]:
nltk.download('stopwords')
corpus_train_esA = procesar(corpus_train_esA, "/content/train_es_cleanA.tsv")
corpus_dev_esA = procesar(corpus_dev_esA, "/content/dev_es_cleanA.tsv")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [251]:
#Leyendo el corpus ya procesado-limpio A
train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[1]].fillna(' ')
y_train_hsA = corpus_train_esA[corpus_train_esA.columns[2]]

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[1]].fillna(' ')
y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[2]]

In [252]:
print( len(X_train_textA), len(y_train_hsA) )

print( len(X_test_textA), len(y_test_hsA) )

4469 4469
500 500


## Primera técnica de procesamiento de texto: CountVectorizer (No utlizada)

In [253]:
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters
    ngram_range=(1,2),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

(4469, 1745)
(500, 1745)


## Segunda técnica de procesamiento de texto: TfidfVectorizer (Utilizada)

In [254]:
tvectorizer = TfidfVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters
    ngram_range=(1,2),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_tvectorized = tvectorizer.transform(X_train_textA).toarray()
print(X_train_tvectorized.shape)

X_test_tvectorized = tvectorizer.transform(X_test_textA).toarray()
print(X_test_tvectorized.shape)

(4469, 1745)
(500, 1745)


# 2. Modelos

### (1) Ejecución de Modelo Multinomial

In [255]:
#Etapa Entrenamiento
clf1 = MultinomialNB() #Modelo
clf1.fit(X_train_tvectorized, y_train_hsA)

#Etapa Prueba
predictions = clf1.predict(X_test_tvectorized)

#Metricas
exactitud_MULTI = accuracy_score(y_test_hsA, predictions)
f1_MULTI = f1_score(y_test_hsA, predictions)
auc_MULTI = roc_auc_score(y_test_hsA, predictions)
print( 'Accuracy',  exactitud_MULTI )
print( 'AUC', f1_MULTI )
print( 'F1', auc_MULTI )

Accuracy 0.772
AUC 0.7135678391959798
F1 0.7586687406831292


### (2) Ejecución de Modelo SVC

In [256]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [257]:
from sklearn.svm import SVC, LinearSVC

#Etapa Entrenamiento
X_train_vectorized = add_feature(X_train_tvectorized, X_train_textA.str.len())
model = SVC(C=100) # con regularización 'C = 100'
model.fit(X_train_vectorized, y_train_hsA)

#Etapa Prueba
X_test_vectorized = add_feature(X_test_tvectorized, X_test_textA.str.len())
predictions = model.predict(X_test_vectorized)

#Metricas
exactitud_svc = accuracy_score(y_test_hsA, predictions)
f1_svc = f1_score(y_test_hsA, predictions)
auc_svc = roc_auc_score(y_test_hsA, predictions)
print( 'Accuracy',  exactitud_svc )
print( 'AUC', f1_svc )
print( 'F1', auc_svc )

Accuracy 0.612
AUC 0.33561643835616434
F1 0.5725905761876985


### (3) Ejecución de Modelo Árboles de Clasificación

In [258]:
from sklearn.tree import DecisionTreeClassifier

#Etapa Entrenamiento
X_train_vectorized = add_feature(X_train_tvectorized, X_train_textA.str.len() )
X_train_vectorized = add_feature(X_train_tvectorized, X_train_textA.str.count('\d') )
arbol = DecisionTreeClassifier(max_depth=2)
arbol.fit(X_train_vectorized,y_train_hsA)

#Etapa Prueba
X_test_vectorized = add_feature(X_test_tvectorized, X_test_textA.str.len())
X_test_vectorized = add_feature(X_test_tvectorized, X_test_textA.str.count('\d'))
predictions = arbol.predict(X_test_vectorized)

#Metricas
exactitud_arbol = accuracy_score(y_test_hsA, predictions)
f1_arbol = f1_score(y_test_hsA, predictions)
auc_arbol = roc_auc_score(y_test_hsA, predictions)
print( 'Accuracy',  exactitud_arbol )
print( 'AUC', f1_arbol )
print( 'F1', auc_arbol )

Accuracy 0.66
AUC 0.4585987261146497
F1 0.6261909391405794


### (4) Ejecución de Modelo KNN

In [259]:
from sklearn.neighbors import KNeighborsClassifier

#Etapa Entrenamiento
X_train_vectorized = add_feature(X_train_tvectorized, X_train_textA.str.len())
X_train_vectorized = add_feature(X_train_tvectorized, X_train_textA.str.count('\d'))
modelo = KNeighborsClassifier(n_neighbors=5)
modelo.fit(X_train_vectorized,y_train_hsA)

#Etapa Prueba
X_test_vectorized = add_feature(X_test_tvectorized, X_test_textA.str.len())
X_test_vectorized = add_feature(X_test_tvectorized, X_test_textA.str.count('\d'))
predictions = modelo.predict(X_test_vectorized)

#Metricas
exactitud_knn = accuracy_score(y_test_hsA, predictions)
f1_knn = f1_score(y_test_hsA, predictions)
auc_knn = roc_auc_score(y_test_hsA, predictions)
print( 'Accuracy',  exactitud_knn )
print( 'AUC', f1_knn )
print( 'F1', auc_knn )

Accuracy 0.654
AUC 0.44372990353697744
F1 0.6194341823838226


# Tabla de comparación

In [260]:
# Construimos la tabla para comparar los métodos
import pandas as pd

tabla = pd.DataFrame()

tabla['Exactitud'] = None
tabla['F1'] = None
tabla['AUC'] = None

tabla.loc['Multinomial'] = [exactitud_MULTI, f1_MULTI, auc_MULTI]
tabla.loc['SVC'] = [exactitud_svc,f1_svc,auc_svc]
tabla.loc['Arboles '] = [exactitud_arbol,f1_arbol,auc_arbol]
tabla.loc['KNN'] = [exactitud_knn,f1_knn,auc_knn]
tabla

,Exactitud,F1,AUC
Multinomial,0.772,0.713568,0.758669
SVC,0.612,0.335616,0.572591
Arboles,0.660,0.458599,0.626191
KNN,0.654,0.443730,0.619434
